In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import h5py
# import skimage
import matplotlib.pyplot as plt
import sys
# Input data files are avahilable in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory


In [3]:
# Any results you write to the current directory are saved as output.

#File Format
#f=h5py.File('../input/food_c101_n1000_r384x384x3.h5','r')
#f=h5py.File('../raw_data/dataset_food/food_c101_n10099_r64x64x3.h5','r')
#print(list(f.keys()))
#print(len(f["category"]))
#print(len(f["category_names"]))
#print(len(f["images"]))

In [4]:
f=h5py.File('/mnt/c/Users/dani_/Downloads/archive/food_c101_n10099_r64x64x3.h5','r')
print(list(f.keys()))
print(len(f["category"]))
print(len(f["category_names"]))
print(len(f["images"]))

['category', 'category_names', 'images']
10099
101
10099


In [4]:
f['images'].shape, f['category'].shape, f['category_names'].shape

((10099, 64, 64, 3), (10099, 101), (101,))

In [5]:
f['category'][0].astype(int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [6]:
# Print Sample Pictures
#print([int(i) for i in f["category"][0]])
#print(f["images"][0].shape)
#fig=plt.figure(figsize=(20,20))
#n=25
#col=5
#for i in range(n):
 #   ax=fig.add_subplot(n/col,col,i+1)
  #  #ax.set_title(f["category_names"][i].decode())
   # ax.imshow(f["images"][i])
#plt.savefig("./sample_show_64x64")

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers
from tensorflow.keras.utils import get_file
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import pathlib
import numpy as np

In [8]:
X = f['images'][1:500]

In [9]:
y = f['category'][1:500]

In [10]:
y = y.astype(int)

In [11]:
from sklearn.model_selection import train_test_split
def load_data():
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    return X_train, X_test, y_train, y_test

In [12]:
# size of each image
IMAGE_SHAPE = (64, 64, 3)

In [13]:
def create_model(input_shape):
    # load MobileNetV2
    model = MobileNetV2(include_top=False,input_shape=input_shape, weights= 'imagenet')
    model.trainable = False
    base_model = model 
    flattening_layer = layers.Flatten()
    dense_layer = layers.Dense(100, activation = 'relu')
    prediction_layer = layers.Dense(101, activation = 'softmax')
    
    model = Sequential([base_model, flattening_layer, dense_layer,prediction_layer ])
    # training the model using adam optimizer
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

In [21]:

# load the data generators
X_train, X_test, y_train, y_test = load_data()
    # constructs the model
model = create_model(input_shape=IMAGE_SHAPE)

from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_acc', mode='max', patience=5, verbose=0, restore_best_weights=True)

history = model.fit(X_train, y_train, 
                    validation_split = 0.2, 
                    epochs=20, 
                    batch_size=32, 
                    callbacks=[es],verbose =0)

In [20]:
model.evaluate(X_test, y_test)

4/4 [==============================] - 1s 69ms/step - loss: 17.6101 - accuracy: 0.0300


[17.610061645507812, 0.029999999329447746]